### Meta

In [1]:
# Autoreload modules without having to restart the notebook kernel.
# hi bilbo
%load_ext autoreload
%autoreload 2


# Plotting code stolen from Georg's notebook.
import matplotlib.pyplot as plt

%matplotlib inline
font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 32}
plt.rc('font', **font)

### Imports

In [ ]:
import sys
sys.path.insert(1, "..\\")


#import markov_clustering as mcl
import networkx as nx

# Personal libraries
import lib.graph
import lib.files
import lib.centrality


In [3]:
# The first network is the largest connected component containing ICP55 with inviable proteins, has edges above 900 and no degree 0 nodes.
n0 = lib.graph.read_weighted_edgelist(lib.files.make_filepath_to_networks("icp55-cc-900-inv.txt"))

In [ ]:
nx.draw(n0)

In [4]:

n0_bridging_centrality = lib.centrality.bridging_centrality(n0,
                         betweenness_approx_nodes = False)

print(n0_bridging_centrality)
print(max(n0_bridging_centrality))


KeyboardInterrupt: 